In [62]:

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import lxml

In [2]:
index_url='https://sipmen.bps.go.id/regsosek/cetak-box/index-generate-box-kab'
first_page=index_url+'?page=1&search='
login_url='https://sipmen.bps.go.id/regsosek/login'

# user login credential
email='pujilestariput@gmail.com'
password="sipmen"
# email="r4ak4a@gmail.com"
# password="r4ak4a"

In [3]:
driver = webdriver.Chrome('./chromedriver')
driver.get(first_page)

time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# simulasi login
if(driver.current_url==login_url):
    wait = WebDriverWait(driver, 1)
    wait.until(EC.element_to_be_clickable((By.ID, "email"))).send_keys(email)
    wait.until(EC.element_to_be_clickable((By.ID, "password"))).send_keys(password)
    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()=' Sign In']"))).click()
    
time.sleep(2)

no_batch = []
kode_sls = []

# halaman index/ halaman pertama
if(driver.current_url==first_page):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup_table = soup.find("table")
    
    table = pd.read_html(str(soup_table), converters ={x:str for x in ['NOMOR BATCH', 'RINCIAN DOKUMEN']})[0]
    # ambil no batch
    no_batches = table.iloc[:,0]

    # untuk setiap nomor batch (kita perlu indexnya)
    for i in range(len(no_batches)):  
    # ambil rincian dokumen, ekstrak ul dulu, trus ektrak li

        ul = soup_table.find_all('ul')[i]
        lis = ul.find_all('li')
        for li in lis:
            no_batch.append(no_batches[i])
            kode_sls.append(li.text.strip())

    #halaman kedua dst
    dataIsAvailable = True
    page=2
    while(dataIsAvailable):

        url = index_url+'?page='+str(page)+'&search='

        driver.get(url)
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        soup_table = soup.find("table")
        table = pd.read_html(str(soup_table), converters ={x:str for x in ['NOMOR BATCH', 'RINCIAN DOKUMEN']})[0]
        # kalau halaman terakhir, maka cuma ada satu row, nilainya 'tidak ditemukan'

        if(len(table.index)<=1):
            dataIsAvailable=False
            # table['KODE WILAYAH'].astype(str)
            # table['NO SLS'].astype(str)
            columns = ['no batch', 'kode sls']
            data = np.array([no_batch, kode_sls])
            table_final = pd.DataFrame(data=data, columns=columns)
            table_final.to_csv('sipmen_batching')
        else:
            # 
            no_batches = table.iloc[:,0]
            # untuk setiap nomor batch (kita perlu indexnya)
            for i in range(len(no_batches)):  
            # ambil rincian dokumen, ekstrak ul dulu, trus ektrak li
                ul = soup_table.find_all('ul')[i]
                lis = ul.find_all('li')
                for li in lis:
                    no_batch.append(no_batches[i])
                    kode_sls.append(li.text.strip())
        page=page+1
        
    driver.close()


C:\Users\ASUS\AppData\Local\Temp\ipykernel_9600\3284722527.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [48]:
no_batchs = table.iloc[:,0]
for no_batch in no_batchs:


,NOMOR BATCH,RINCIAN DOKUMEN,AKSI
0,15060001,15 06 010 010 000200 15 06 010 010 000300 15 0...,NaN
1,15060002,15 06 010 011 000200 15 06 010 011 000300 15 0...,NaN
2,15060003,15 06 010 012 002100 15 06 010 012 002200 15 0...,NaN
3,15060004,15 06 010 013 001600 15 06 010 013 001500 15 0...,NaN
4,15060005,15 06 010 014 001800 15 06 010 014 001700 15 0...,NaN
5,15060006,15 06 010 015 002600 15 06 010 015 001000 15 0...,NaN
6,15060007,15 06 010 016 002100 15 06 010 016 002000 15 0...,NaN
7,15060008,15 06 010 017 000100 15 06 010 017 000200 15 0...,NaN
8,15060009,15 06 010 018 000100 15 06 010 018 000200 15 0...,NaN
9,15060010,15 06 011 001 000800 15 06 011 001 000700 15 0...,NaN


In [54]:
no_batches = table.iloc[:,0]
no_batches

for i in range(len(no_batches)):
    print('index', i,'elemen :',no_batches[0])

index 0 elemen : 15060001
index 1 elemen : 15060001
index 2 elemen : 15060001
index 3 elemen : 15060001
index 4 elemen : 15060001
index 5 elemen : 15060001
index 6 elemen : 15060001
index 7 elemen : 15060001
index 8 elemen : 15060001
index 9 elemen : 15060001


In [32]:
table = pd.read_html(str(soup_table), converters ={x:str for x in ['NOMOR BATCH', 'RINCIAN DOKUMEN']})[0]
    # skip baris pertama karna bernilai NaN
# table = table.iloc[1:, :]
table

,NOMOR BATCH,RINCIAN DOKUMEN,AKSI
0,15060001,15 06 010 010 000200 15 06 010 010 000300 15 0...,NaN
1,15060002,15 06 010 011 000200 15 06 010 011 000300 15 0...,NaN
2,15060003,15 06 010 012 002100 15 06 010 012 002200 15 0...,NaN
3,15060004,15 06 010 013 001600 15 06 010 013 001500 15 0...,NaN
4,15060005,15 06 010 014 001800 15 06 010 014 001700 15 0...,NaN
5,15060006,15 06 010 015 002600 15 06 010 015 001000 15 0...,NaN
6,15060007,15 06 010 016 002100 15 06 010 016 002000 15 0...,NaN
7,15060008,15 06 010 017 000100 15 06 010 017 000200 15 0...,NaN
8,15060009,15 06 010 018 000100 15 06 010 018 000200 15 0...,NaN
9,15060010,15 06 011 001 000800 15 06 011 001 000700 15 0...,NaN


10

In [11]:
# run it to read file
table = pd.read_csv('sipmen_penerimaan_dari_koseka')
table
table['KODE WILAYAH']=table['KODE WILAYAH'].astype(str)


In [12]:
1506011001
1506011002
1506011003
1506011004
1506011005
1506011006
1506011007

1506011007

In [13]:
# tables['KODE WILAYAH'].astype(str)
# temp = table['KODE WILAYAH'].astype(str)
# x = temp.replace('.0', '')
# print(x)


In [14]:
# filter berdasar
desaBerbak = [{
    "kode_wilayah":'1506041001',
    "nama":'Simpang'
},
{
    "kode_wilayah":'1506041002',
    "nama":'Rawasari'
},
{
    "kode_wilayah":'1506041003',
    "nama":'Rantau Makmur'
},
{
    "kode_wilayah":'1506041004',
    "nama":'Telaga Limo'
},
{
    "kode_wilayah":'1506041005',
    "nama":'Sungai Rambut'
},
{
    "kode_wilayah":'1506041006',
    "nama":'Rantau Rasau'
}
]
# desaBerbak = ['1506041001', '1506041002', '1506041003', '1506041004', '1506041005', '1506041006']
# len(table.loc[table['KODE WILAYAH'] =='1506041001'])
desaBerbakDf = pd.DataFrame(desaBerbak)
desaBerbakDf

desaMenlu = [{
    "kode_wilayah":'1506011001',
    "nama":'Bukit Tempurung'
},
{
    "kode_wilayah":'1506011002',
    "nama":'Sungai Toman'
},
{
    "kode_wilayah":'1506011003',
    "nama":'Simpang Tuan'
},
{
    "kode_wilayah":'1506011004',
    "nama":'Pematang Rahim'
},
{
    "kode_wilayah":'1506011005',
    "nama":'Sinar Wajo'
},
{
    "kode_wilayah":'1506011006',
    "nama":'Sungai Beras'
},
{
    "kode_wilayah":'1506011007',
    "nama":'Mencolok'
}
]

desaMenluDf = pd.DataFrame(desaMenlu)

In [15]:
# filter buat berbak
# berbakEntried = table.loc(table['KODE WILAYAH'] in desaBerbakDf['kode_wilayah'])
berbakEntried = table[table['KODE WILAYAH'].isin(desaBerbakDf['kode_wilayah'])]
menluEntried = table[table['KODE WILAYAH'].isin(desaMenluDf['kode_wilayah'])]

In [16]:
# [item for item in desaBerbakDf['kode_wilayah']]
menluEntriedPerDesa = menluEntried['KODE WILAYAH'].value_counts()
agregat = {
    'jumlah':menluEntriedPerDesa.to_list(),
    'kode_wilayah':menluEntriedPerDesa.index.to_list()
}
agregatDf = pd.DataFrame(agregat)

In [17]:
final = pd.merge(agregatDf, desaMenluDf)

In [18]:
final

,jumlah,kode_wilayah,nama
0,25,1506011006,Sungai Beras
1,24,1506011005,Sinar Wajo
2,16,1506011003,Simpang Tuan
3,14,1506011004,Pematang Rahim
4,10,1506011007,Mencolok
5,9,1506011002,Sungai Toman
6,8,1506011001,Bukit Tempurung
